In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import scienceplots

import time
import math
import os
import random
from functools import partial
from decimal import Decimal
from collections import Counter
import numpy as np
import scipy.io as sio
import pysindy as ps
from tqdm import trange

from utils import *
from skimage.restoration import estimate_sigma
import bm3d
from solvel0 import solvel0
from best_subset import backward_refinement, brute_force_all_subsets
from UBIC import *
from kneed import KneeLocator
from bayesian_model_evidence import log_evidence

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

In [2]:
X_pre = np.load("./Cache/X_weak_RD_2025.npy").astype(np.float32)
y_pre = np.load("./Cache/y_weak_RD_2025.npy").astype(np.float32)
# Ground truth
ground_indices_u = np.array((0, 5, 6, 7, 8, 11, 17))
ground_coeff_u = np.array([1.000,-1.000,1.000,-1.000,1.000,0.100,0.100])
ground_indices_v = np.array((1, 5, 6, 7, 8, 12, 18))
ground_coeff_v = np.array([1.000,-1.000,-1.000,-1.000,-1.000,0.100,0.100])

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn import covariance
from abess import LinearRegression as AbessLinearRegression
from knockpy import KnockoffFilter, knockoff_stats, knockoffs
from scipy import stats
from statsmodels.stats.multitest import multipletests
### extra ###
from c2st.check import c2st # https://github.com/psteinb/c2st

X_scale = StandardScaler().fit_transform(X_pre)
X_pre_top = X_scale.copy()
y_pre = (y_pre-y_pre.mean())/y_pre.std()
y_pre = y_pre[:, 0:1]

# lr = SCO(path_type='gs', sparsity=10, ic_method='LinearSIC')
lr = AbessLinearRegression(path_type='gs', s_max=12, fit_intercept=False, cv=5, screening_size=0)
fstat = knockoff_stats.Eli5PIStatistic(model=lr, n_iter=10)
# fstat = knockoff_stats.ShapStatistic(model=lr)
kfilter = KnockoffFilter(ksampler='gaussian', fstat=fstat, knockoff_kwargs={'method':'ci'})
# kfilter = KnockoffFilter(ksampler='gaussian', fstat='lasso', knockoff_kwargs={'method':'ci'})

fdr = 0.2
rejections = []
np.random.seed(1234)
for _ in trange(50):
    rejection = kfilter.forward(X=X_pre_top, y=y_pre.flatten(), fdr=fdr, shrinkage="ledoitwolf", recycle_up_to=0.5)
    rejection = set(np.where(rejection == 1)[0])
    if len(rejection) > 0:
        rejections.append(rejection)

alibi is not installed in the environment.


100%|██████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:57<00:00,  1.15s/it]


In [4]:
assert set(ground_indices_u).issubset(biggest_superset(rejections))
rejections = np.array(sorted(biggest_superset(rejections)))
X_pre_top = X_pre_top[:, rejections]

In [5]:
alpha = 0.05
while True:
    non_null_indices, shap_values = shap_model_selection(X_pre_top, y_pre)
    rejections = rejections[non_null_indices]
    X_pre_top = X_pre_top[:, non_null_indices]
    print(abs(shap_values).mean(axis=0))
    
    j = len(rejections)-1
    classifier_confidences = []
    for _ in trange(50):
        Xk = knockoffs.GaussianSampler(X_pre_top, Sigma=covariance.LedoitWolf().fit(X_pre_top).covariance_, 
                                       method='ci').sample_knockoffs()
        Xn = X_pre_top.copy()
        Xn[:, j] = Xk[:, j]
        
        swap_explainer = shap.explainers.Linear(linear_model.LinearRegression(fit_intercept=False).fit(Xn, y_pre),
                                                Xn)
        swap_shap_values = swap_explainer(Xn).values
        
        classifier_confidences.append(c2st(shap_values[:, j:j+1], swap_shap_values[:, j:j+1], clf=linear_model.LogisticRegression()))

    classifier_confidences = np.array(classifier_confidences)
    pv = stats.wilcoxon(classifier_confidences-0.51, alternative='greater').pvalue
    
    decision = classifier_confidences.mean() >= 0.51
    decision = pv < alpha
    print("binary classifier's acc:", classifier_confidences.mean())
    print("P-value:", pv)

    if decision:
        break
    else:
        rejections = rejections[:j]
        X_pre_top = X_pre_top[:,:j]

[0.9723034  0.6119583  0.5902573  0.26487362 0.25998566 0.05612521
 0.05611704 0.03836295]


100%|██████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:17<00:00,  2.89it/s]

binary classifier's acc: 0.539767671364943
P-value: 8.881784197001252e-16


In [6]:
from mbic import mbic, mbic2, ebic
best_subsets = solvel0(X_pre_top, y_pre, miosr=True, refine=True)
best_subsets = [tuple(best_subsets[-1][_] for _ in bs) 
                for bs in brute_force_all_subsets(X_pre_top[:, best_subsets[-1]], y_pre)[1]]

  0%|                                                                                                | 0/8 [00:00<?, ?it/s]

Set parameter Username
Academic license - for non-commercial use only - expires 2026-04-04


100%|████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 24.94it/s]


In [7]:
# TODO: Calculate post_means for ARDRegression as well (Implement the ard_uncertainties function)
ard_uns = []
threshold_lambda = 5e5 # must pass assert 
for bs in best_subsets:
    ard = linear_model.ARDRegression(fit_intercept=False, 
                                     compute_score=True,
                                     threshold_lambda=threshold_lambda)
    ard.fit(X_pre_top[:, bs], y_pre.ravel())
    print(len(bs), end=', ')
    assert len(bs) == len(np.nonzero(ard.coef_)[0])
    pde_uncert = np.sqrt(np.diag(ard.sigma_)).sum()/abs(ard.coef_).sum()
    ard_uns.append(pde_uncert)
ard_uns = np.array(ard_uns)
ard_uns = ard_uns/min(ard_uns)
ard_uns

1, 2, 3, 4, 5, 6, 7, 8, 

array([ 1.       ,  1.9191405,  3.6291192,  4.768603 , 10.173801 ,
       26.454456 ,  6.2694693, 16.009884 ], dtype=float32)

In [8]:
tau = 3
verbose = True
# scale = 1 <- generalized UBIC
scale = np.log(len(y_pre))
per = 75 # 80

post_means, b_bics, b_uns = baye_uncertainties(best_subsets, (X_pre_top, y_pre), 
                                               u_type='cv1', take_sqrt=True, 
                                               ridge_lambda=0, 
                                               threshold=0)
# b_uns = ard_uns # USE ard_uns INSTEAD
predictions = X_pre_top@post_means
print(b_bics)
print(b_uns)
b_bics = np.array(b_bics)
max_complexity = len(b_bics)
complexities = np.arange(max_complexity)+1
d_complexities = complexities[decreasing_values_indices(b_bics)]
d_bics = b_bics[decreasing_values_indices(b_bics)]
slopes = np.diff(d_bics)/(np.diff(d_complexities)*d_bics[:-1])
try:
    thres = np.percentile(np.abs(slopes), per)
    thres = math.ceil(sci_format(thres)[0])*10**sci_format(thres)[1]
except IndexError:
    thres = 1/40
min_thres = 1/40
thres = max(thres, min_thres)
print("threshold:", thres)

lower_bounds = []
for k, efi in enumerate(best_subsets):
    # assert len(efi) == np.count_nonzero(post_means[:, k:k+1])
    com = len(efi)
    lower_bound = 2*np.abs(log_like_value(predictions[:, k:k+1], y_pre))-np.log(len(y_pre))*com
    lower_bounds.append(lower_bound)

last_lam = np.log10(max(lower_bounds/(b_uns*scale)))
print("max_lam:", last_lam)
delta = last_lam/tau
now_lam = last_lam-delta
last_ubic = UBIC(b_bics, b_uns, len(y_pre), hyp=10**last_lam, scale=scale)
last_bc = np.argmin(last_ubic)
bc_seq = [last_bc]
while now_lam >= 0:
    now_ubic = UBIC(b_bics, b_uns, len(y_pre), hyp=10**now_lam, scale=scale)
    now_bc = np.argmin(now_ubic)
    
    diff_com = now_bc-last_bc
    diff_bic = b_bics[now_bc]-b_bics[last_bc]
    imp = np.nan
    if diff_com != 0:
        imp = abs(diff_bic/(b_bics[last_bc]*diff_com))
    
    if verbose:
        print(min(last_bc, now_bc), '<--->', max(last_bc, now_bc), 
              np.nan_to_num(imp, nan=np.inf))
    
    if (diff_com > 0 and (diff_bic > 0 or imp < thres)) or \
        (diff_com < 0 and diff_bic > 0 and imp > thres):
        break
    
    last_lam = now_lam
    now_lam = round(last_lam-delta, 8)
    last_ubic = now_ubic
    last_bc = now_bc
    if last_bc not in bc_seq:
        bc_seq.append(last_bc)

# best_bc = knee(range(len(last_ubic)), last_ubic, 0.95, 'linear', direction='decreasing')
best_bc = knee_finder(last_ubic)
if best_bc == 0 and last_bc != 0 and b_bics[last_bc] < b_bics[0] and \
                                    abs((b_bics[last_bc]-b_bics[0])/(b_bics[0]*last_bc)) > thres:
    best_bc = knee(range(1, len(last_ubic)), last_ubic[1:], 0.95, 'linear')
if best_bc < last_bc and abs((b_bics[last_bc]-b_bics[best_bc])/(b_bics[best_bc]*(last_bc-best_bc))) > thres:
    best_bc = last_bc
    
last_lam = round(last_lam, 8)
last_lam, last_ubic, last_bc, best_bc

[-42862.96107663496, -43563.69859410328, -44094.79681855538, -47271.912678188506, -47304.30902705898, -51075.080848937054, -71125.40145125426, -71195.58896801552]
[ 1.          1.91965858  3.63122435  4.76864352 10.24666284 26.44636571
  6.26685123 15.99791453]
threshold: 0.08
max_lam: 3.667806489824039
0 <---> 6 0.10989457122855913
6 <---> 6 inf
6 <---> 6 inf


(-0.0,
 array([-42853.75073626, -43546.01788516, -44061.35200637, -47227.99184825,
        -47209.93377465, -50831.50081914, -71067.68161837, -71048.24272998]),
 6,
 6)

### Selective inference ###

In [9]:
from selective_inference import sfs_si, stepwise_selective_inference, subset_fdr
import fpsample
from dppy.finite_dpps import FiniteDPP

from si4pipeline import ( 
                        construct_pipelines, 
                        extract_features, 
                        initialize_dataset, 
                        intersection, 
                        lasso, 
                        marginal_screening, 
                        stepwise_feature_selection, 
                        union, 
                        PipelineManager
                        )

In [10]:
n_samples = 100*(best_bc+1)
max_fdr = 0.05
for bs in best_subsets:
    fdrs = []
    for _ in range(len(y_pre)//n_samples):
        X_test = X_pre_top[:, bs]
        y_test = y_pre.ravel()
        
        np.random.seed(random.randint(0, 100))
        # sample_indices = sorted(set([np.random.randint(len(y_pre)) for _ in range(n_samples)]))
        sample_indices = fpsample.bucket_fps_kdline_sampling(X_test, n_samples=n_samples, h=5) # Farthest Point Sampling (FPS) is better!!!
        X_test = X_test[sample_indices]; y_test = y_test[sample_indices]
        # FPS + k-DPP
        # DPP = FiniteDPP('likelihood', **{'L': X_test.dot(X_test.T)})
        # DPP.flush_samples()
        # for _ in range(n_samples//(len(bs))):
        #     DPP.sample_exact_k_dpp(size=len(bs))
        # sample_indices = np.unique(np.ravel(DPP.list_of_samples))
        # X_test = X_test[sample_indices]; y_test = y_test[sample_indices]
        
        manager = stepwise_selective_inference(support_size=X_test.shape[1])
        M, p_list = manager.inference(X_test, y_test, np.std(y_test))
        # print(M, p_list, subset_fdr(p_list) < 0.01, np.array(p_list) < 0.05)
        fdrs.append(subset_fdr(p_list))
        
    fdrs = np.array(fdrs)
    print(Counter(fdrs < max_fdr), fdrs.mean())

Counter({True: 14}) 0.0
Counter({False: 14}) 0.421671494422755
Counter({False: 14}) 0.61420880445064
Counter({False: 14}) 0.7433967279135411
Counter({False: 14}) 1.122993187359406
Counter({False: 14}) 0.6373181691376487
Counter({True: 14}) 0.03973898690764753
Counter({False: 14}) 0.8285702386905577
